In [39]:
import pandas as pd
import numpy as np
import datetime

import re
import itertools

from konlpy.tag import Mecab

mec = Mecab("C:/mecab/mecab-ko-dic")

path = "./물금고등학교-익명의-숲_생활편/"

In [40]:
df_rating = pd.read_csv(path + 'data/ratings.csv')
df_rating.dropna(inplace=True)

clean = re.compile(r'[^가-힣ㄱ-ㅎㅏ-ㅣ ]')
df_rating['document'] = df_rating['document'].apply(lambda x: clean.sub('', x))
df_rating['document'] = df_rating['document'].apply(lambda x: ''.join(ch for ch, _ in itertools.groupby(x)))
df_rating.drop_duplicates(subset=['document'], inplace=True)

In [41]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(df_rating['document'], df_rating['label'], random_state=12)

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

tfv = TfidfVectorizer(tokenizer=mec.morphs, token_pattern='(?u)\b\w\w+\b', ngram_range=(1, 2), min_df=10, max_df=0.9)
tfv.fit(train_x)
tfv_train_x = tfv.transform(train_x)

clf = LogisticRegression(random_state=12)
params = {'C': [1, 3, 5, 7, 9]}
grid_cv = GridSearchCV(clf, param_grid=params, cv=4, scoring='accuracy', verbose=1)
grid_cv.fit(tfv_train_x, train_y)

C:\Users\Heoshin\miniconda3\envs\exhibition\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


Fitting 4 folds for each of 5 candidates, totalling 20 fits


C:\Users\Heoshin\miniconda3\envs\exhibition\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\Heoshin\miniconda3\envs\exhibition\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/

GridSearchCV(cv=4, estimator=LogisticRegression(random_state=12),
             param_grid={'C': [1, 3, 5, 7, 9]}, scoring='accuracy', verbose=1)

In [47]:
a = [
    '좋아',
    '행복해',
    '짜증나'
]
review = tfv.transform(a)
grid_cv.best_estimator_.predict(review)

<3x34937 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Row format>

In [45]:
import pickle
import joblib

joblib.dump(grid_cv, 'model/emotionModel.plk')
saved_tfv = pickle.dumps(tfv)
joblib.dump(tfv, 'model/tfv.plk')


['model/tfv.plk']

In [51]:
saved_grid_cv = pickle.dump(grid_cv, open('model/emotionModel.plk', 'wb'))
saved_tfv = pickle.dump(tfv, open('model/tfv.plk', 'wb'))

None
